### 需要先設置SPARK_HOME 
Initialization of the spark context
Note that we can set things like driver memory etc.
If launcher._spark_home is not set it will default to looking at the SPARK_HOME environment variable.

In [1]:
%%init_spark
launcher.num_executors = 1
launcher.executor_cores = 1
launcher.driver_memory = '2g'

### 如果想在Spark運行時中使用其他jar，使用launcher.jars和/或launcher.packages設置，後者使用 Maven "GAV" coordinates：

* launcher.jars = ["/some/local/path/to/a/file.jar"]
* launcher.packages = ["com.acme:super:1.0.1"]

In [2]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://Zoxas:4040
SparkContext available as 'sc' (version = 2.4.0, master = local[*], app id = local-1544278180613)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@45b9b29d


In [3]:
spark.version

res1: String = 2.4.0


### scala 基本補充說明

* Scala 類似Java 支持單行和多行註釋 // /**/
* 類Any是Scala類層次結構的根，是一個抽像類。Scala中的每個類都直接或間接從這個類繼承。AnyVal和 AnyRef擴展任何類型。
* Scala 使用import 關鍵字引用 import java.awt._ --> 引用全部成員
  1. 在要導入package後使用"{}",指定要導入的成員，並使用"=>"重命名;如果在一個package下要導入多個成員，則使用逗號分隔.
  2. Scala能導入object class的成員，也能導入運行時對象實例的成員
  3. import 可以位於表達式的能放的任意位置  
    
   
---  
* 整型字面量用於Int 類型，如果表示Long，可以在數字後面添加 L 或者小寫l 作為後綴
* 如果浮點數後面有f或者F後綴時，表示這是一個Float類型，否則就是一個Double類型的(D 或 d)
* Unit	表示無值，和其他語言中void等同。用作不返回任何結果的方法的結果類型。Unit只有一個實例值，寫成()。
  

### scala 變量宣告

* var VariableName : DataType [=  Initial Value]
* val VariableName : DataType [=  Initial Value]

In [4]:
// 不可變 變量關鍵字用 val 聲明 -> 且需要初始化
val x = 10
print(x*x)

// array用 val宣告  element本身還是可變
val array : Array[String] = new Array(5) 
      // array = new Array(2) -> 會出錯 
      array(0) = "Hello"  
     println( array ) 

100[Ljava.lang.String;@69c34c65


x: Int = 10
array: Array[String] = Array(Hello, null, null, null, null)


In [5]:
//var 是可變變量
class Person(val name: String, var age: Int) 

val p = new Person( "Dean Wampler" , 29) 
println(p.name) 
println(p. age ) 
p.age = 30 
println(p.age ) 

Dean Wampler
29
30


defined class Person
p: Person = Person@3fa0313d
p.age: Int = 30


### scala code block { }

In [6]:
// 結果是在{ }計算的最後一行
// 變量定義也可以用{ }

val x3:String= {
    val d = new java.util.Date()
    d.toString()
}

x3: String = Tue Nov 20 23:29:56 CST 2018


### scala 格式化字符串
* 單引號表示：char字符 || 雙引號表示：string字符
* Scala提供了三種創新的字符串插值方法

In [7]:
//s字符串插值 -> 在任何字符串前加上S，就可以直接在字符串中使用變量了
val name="James"
println(s"Hello,$name")

println(s"1+1=${1+1}") //任何表達式都可以嵌入到${}中

Hello,James
1+1=2


name: String = James


In [8]:
//f插值 -> 格式化字串
val height=1.9d
val name="James"
println(f"$name%s is $height%2.2f meters tall")

James is 1.90 meters tall


height: Double = 1.9
name: String = James


In [9]:
//raw插值 -> 不做編碼
var s = raw"a\nb"

s: String = a\nb


In [10]:
//在code中使用多行字符串時，要使用String.stripMargin縮進子字符串以進行正確的代碼格式化
//它刪除子字符串中的所有空格，直到並包括垂直條|的第一次出現
def hello(name: String) = s"""Welcome!
       Hello, $name! 
       *(Star!!) 
       |Hi. 
       |whitespace. """.stripMargin  

hello("Roy")

hello: (name: String)String
res6: String =
"Welcome!
       Hello, Roy!
       *(Star!!)
Hi.
whitespace. "


### scala symbol 類型
* 由 單引號'，後跟一個或多個數字，字母或下劃線（“_”）
* 下面接著介紹Symbol類型的兩個特點。
  1. **節省 memory**  
    在Scala中，Symbol類型的對像是被拘禁的(interned)，任意的同名symbols都指向同一個Symbol對象，避免了因冗餘而造成的開銷。  
    而對於String類型，只有編譯時確定的字符串是被拘禁的(interned)。    
  2. **快速比較**  
    任意的同名symbols都指向同一個Symbol對象，而不同名的symbols一定指向不同的Symbol對象，所以symbols對象之間可以使用操作符==快速地進行相等性比較  
      
      
* Symbol類型一般用於快速比較，例如用於Map類型：  
    Map< Symbol, Data>,根據一個Symbol對象，可以快速查詢相應的Data,   
    Map< String, Data>的查詢效率則低很多。  

In [11]:
val s = 'aSymbol

//输出true
println( s == 'aSymbol)

//输出true
println( s == Symbol("aSymbol"))

//输出false
print(s == "aSymbol")

true
true
false

s: Symbol = 'aSymbol


## scala 特殊用法 箭頭 ->, <-, => 底線_  

### 符號 <- 用在 for 迴圈, 遍歷集合(生成器表達式)

In [83]:
for (i <- 1 to 10) {print(i)}

println("\n---")

//*****************************************************************
for( a <- 0 to 2; b <- 0 to 1){ //迭代給定區間所有的可能值
 println( "Value of a: " + a );
 println( "Value of b: " + b );
}

println("\n---")
//*****************************************************************
val numList = List(1,2,3,4,5,6);
for( x <- numList ){
 println( "Value of x: " + x );
}

println("\n---")
//*****************************************************************
for( y <- numList //循環過濾(又叫約束)，過濾我們想要保留的元素
   if y != 3; if y < 4 ){
 println( "Value of y: " + y );
}

println("\n---")
//*****************************************************************
var retVal = for{ a <- numList 
                if a != 3; if a < 8
              }yield a //yield 會把當前的元素記下來，保存在集合中，循環結束後將返回該集合

print(retVal)
println("\n---")
//*****************************************************************
val books = List("Scala", "Groovy", "Java", "SQL", "CSS")
//Scala可以在for表達式的第一部分中定義可用於後面表達式的值
for{book <- books //要緊接著大括號
    bookVal = book.toUpperCase()
}println(bookVal)



12345678910
---
Value of a: 0
Value of b: 0
Value of a: 0
Value of b: 1
Value of a: 1
Value of b: 0
Value of a: 1
Value of b: 1
Value of a: 2
Value of b: 0
Value of a: 2
Value of b: 1

---
Value of x: 1
Value of x: 2
Value of x: 3
Value of x: 4
Value of x: 5
Value of x: 6

---
Value of y: 1
Value of y: 2

---
List(1, 2, 4, 5, 6)
---
SCALA
GROOVY
JAVA
SQL
CSS
ScalaGroovyJavaSQLCSS

numList: List[Int] = List(1, 2, 3, 4, 5, 6)
retVal: List[Int] = List(1, 2, 4, 5, 6)
books: List[String] = List(Scala, Groovy, Java, SQL, CSS)


### 符號 -> 常用在 map 映射 或是 創建 tuple(下一節會介紹) 

In [4]:
val colors = Map(
    "red" ->  "#FF0000", 
    "azure" ->  "#F0FFFF", 
    "peru" ->  "#CD853F") 

println("colors  key: " + colors.keys ) 
println("colors values : " + colors.values ) 
println("\n---")

// 用 += 加入元素
var nums: Map[Char, Int] = Map() 
println( "check nums Empty : " + nums.isEmpty )
nums+= ( 'J' -> 5 ) 
println("check colors Empty : " + colors.isEmpty ) 
println("\n---")

//使用++運算符或Map.++()方法來連接兩個Map，Map合併時會移除重複的key
val colors1 = Map ( "red" -> "#error" , "azure" -> "#F0FFFF" , "peru" -> "#CD853F" ) 
val colors2 = Map ( "blue" -> "#0033FF" , "yellow" -> "#FFFF00" , "red" -> "#FF0000") 
var color_map = colors1 ++ colors2 
println ( "colors1 ++ colors2 : " + color_map ) 
println("\n---")


//foreach 循環輸出 Map中的 keys 和values
colors.keys.foreach{ i =>  
print( "Key = " + i )
println(" Value = " + colors(i))}
println("\n---")

//Map.contains方法來查看Map中是否存在指定的Key
if(colors1.contains ("red")){ println ( "the value of red is :" + color_map("red")) }  

colors  key: Set(red, azure, peru)
colors values : MapLike(#FF0000, #F0FFFF, #CD853F)

---
check nums Empty : true
check colors Empty : false

---
colors1 ++ colors2 : Map(blue -> #0033FF, azure -> #F0FFFF, peru -> #CD853F, yellow -> #FFFF00, red -> #FF0000)

---
Key = red Value = #FF0000
Key = azure Value = #F0FFFF
Key = peru Value = #CD853F

---
the value of red is :#FF0000


colors: scala.collection.immutable.Map[String,String] = Map(red -> #FF0000, azure -> #F0FFFF, peru -> #CD853F)
nums: Map[Char,Int] = Map(J -> 5)
colors1: scala.collection.immutable.Map[String,String] = Map(red -> #error, azure -> #F0FFFF, peru -> #CD853F)
colors2: scala.collection.immutable.Map[String,String] = Map(blue -> #0033FF, yellow -> #FFFF00, red -> #FF0000)
color_map: scala.collection.immutable.Map[String,String] = Map(blue -> #0033FF, azure -> #F0FFFF, peru -> #CD853F, yellow -> #FFFF00, red -> #FF0000)


### 第三個符號 => 匿名函數 或是是用在模式匹配  

In [87]:
/*(函數參數) => 函數本體 */
val l = List(1,2,3) 
var ll = l.map(x => x*x) 

//兩者等價
val f1: (Int,String) => String = (i, s) => s+i 
val f2: Function2[Int,String,String] = (i, s) => s+i 
println(f1(1,"a"))
println(f2(1,"v"))



//對於右箭頭，還有一個地方用到就是匹配模式語句case與後面表達式的分隔符
val x: Int = 10
x match { 
  case 0 => "zero"
  case 1 => "one"
  case 2 => "two"
  case n if n > 5 => "n_test"
  case _ => "many"
} 

//測試用 match 類型
val anyList= List(1, "A" , 2, 2.5, 'a' )
def test(in: Any) = in match { 
    case s: String => "String, length "+s.length 
    case i: Int if i > 0 => "Natural Int" 
    case i: Int => "Another Int" 
    case a: AnyRef => a.getClass.getName 
    case _ => "null" 
}

for(item <- anyList){
    println(test(item))
}

a1
v1
Natural Int
String, length 1
Natural Int
java.lang.Double
java.lang.Character


l: List[Int] = List(1, 2, 3)
ll: List[Int] = List(1, 4, 9)
f1: (Int, String) => String = <function2>
f2: (Int, String) => String = <function2>
x: Int = 10
anyList: List[Any] = List(1, A, 2, 2.5, a)
test: (in: Any)String


In [93]:
// 進一步 說明 => 符號可以看做是創建函數實例的語法糖。
// 例如：A => T，A,B => T表示一個函數的輸入參數類型是“A”，“A,B”，返回值類型是 T

var increase = (x: Int) => x + 1  
increase(10)


//定義函數 f：輸入參數是整數類型，返回值是字符串 -> 想像成 schema
//等式右邊是函數 myInt會被映射為"The value of myInt is: " + myInt.toString()
val f: Int => String = myInt => "The value of myInt is: " + myInt.toString()
println(f(3))

//定義了一個函數d,參數類型是Int=>Int的函數,返回值根據上下文推算是Int
//返回值: 發現沒有,返回值是x(2),它調用了傳入函數。結果自然就是6了
def d(x: (Int) => Int) = x(2);
println(d((x: Int) => x * 4));




//換設置2個值。
def c(x: (Int, Int) => Int) = x(2, 3);
println(c((x: Int, y: Int) => x * y * 3));


//參考 http://orchome.com/401

The value of myInt is: 3
8
18


increase: Int => Int = <function1>
f: Int => String = <function1>
d: (x: Int => Int)Int
c: (x: (Int, Int) => Int)Int


In [5]:
/* 在scala中，符號“_”相當於java中的通配符“*”。這個有很多的含義*/
//:_*作為一個整體，告訴編譯器你希望將某個參數當作參數序列處理
// 簡單來說 這個標註告訴編譯器把集合的每個元素當作參數，而集合自己不是當作單一的參數 
val x: Seq[Int] = 1 to 5
List(x: _*)

def echo(args: String*) = for (arg <- args) println(arg)
val arr = Array("What's", "up", "doc?")
echo(arr:_*)

println("\n----")

//代指一個集合中的每個元素
val a = List(1,2,3,4,5,6);
a.filter(_%2==0).map(2*_)

What's
up
doc?

----


x: Seq[Int] = Range(1, 2, 3, 4, 5)
echo: (args: String*)Unit
arr: Array[String] = Array(What's, up, doc?)
a: List[Int] = List(1, 2, 3, 4, 5, 6)
res3: List[Int] = List(4, 8, 12)


### scala 元組
* 元組是具有相同或不同類型的兩個或更多個值的有序容器。
* 我們可以通過兩種方式創建一個元組：
    1. 通過用逗號分隔的值寫入值，並用一對括號括起來
    2. 通過使用關係運算符-> ex: 1 -> "one" 

In [8]:
//與列表一樣，元組也是不可變的，但與列表不同的是元組可以包含不同類型的元素
val t1: (Int,String) = (1, "two") 

//Scala庫包括用於將 N個項分組的TupleN類  N可到22
val t2: Tuple2[Int,String] = (1, "two") 
val t3: Tuple3[Int,String,Double] = (1, "two", 3.0)

t1: (Int, String) = (1,two)
t2: (Int, String) = (1,two)
t3: (Int, String, Double) = (1,two,3.0)


In [9]:
//表達式 t._n從元組t中檢索第 n 個項，從一個開始，不是零。
println( "Print the whole tuple: " + t3 )
println( "Print the first item: " + t3._1 )
println( "Print the second item: " + t3._2 )
println( "Print the third item: " + t3._3 )

//我們可以在兩個值之間使用“箭頭運算符”

Print the whole tuple: (1,two,3.0)
Print the first item: 1
Print the second item: two
Print the third item: 3.0


In [18]:
/*與列表和數組不同，沒有辦法用for迭代元組中的元素 
p.s.但可以用 Tuple.productIterator()  方法來迭代輸出元組的所有元素 */
t3.productIterator.foreach{ i => println ( "Value = " + i )}

//可以使用  Tuple.toString()  方法將元組的所有元素組合成一個字符串
println ( "string output: " + t3.toString())

//可以使用  Tuple2.swap  方法來交換元組的元素
 println ( "tuple swap: " + t2.swap )

Value = 1
Value = two
Value = 3.0
string output: (1,two,3.0)
tuple swap: (two,1)


### Option和Null 和 Nothing 和 Nill 類型
* 任何時候，如果想使用 null ，請用 Option  
  
---  
* option可以看作是一個容器,容器的size是1或0。 Size為1的時候就是一個Some[A](x: A),size為0的時候就是一個None
    1. scala推薦在可能返回空的方法使用Option[X]作為返回類型。如果有值就返回Some[x]，否則返回None
    2. 獲得Option後，可以使用get獲得包含的值，或者使用getOrElse獲得默認值如果isEmpty為true  
   
---        
* Null可以賦值給所有的引用類型(AnyRef)，不能賦值給值類型，這個java的語義是相同的。null是Null的唯一對象。     
  
---    
* Nothing是所有類型的子類，也是Null的子類。Nothing沒有對象，但是可以用來定義類型。例如，如果一個方法拋出異常，則異常的返回值類型就是Nothing      
    
---    
* Nil是一個空的List，定義為List[Nothing]

In [34]:
//Map.get方法返回一個 Option [T] ，在這種情況下 T 是String
val stateCapitals = Map( 
    "Alabama" -> "Montgomery" ,                         
    "Alaska"   -> "Juneau" ,                         
    "Wyoming" -> "Cheyenne" )

//如果 Option 是 Some ，則 Some.get 返回值。
println( "Get the capitals wrapped in Options:" ) 
println( "Alabama: " + stateCapitals.get( "Alabama" ) ) 
println( "Wyoming: " + stateCapitals.get( "Wyoming" ) ) 
println( "Unknown: "+ stateCapitals.get( "Unknown" ) )

println("\n---")

//如果 Option 實際上是 None ，那麼 None.get 將拋出一個 NoSuchElementException 異常。
//getOrElse返回 Option 中的值，如果它是一個 Some 實例，或者返回傳遞給 getOrElse 的參數
println( "Get the capitals themselves out of the Options:" ) 
println( "Alabama: " + stateCapitals.get( "Alabama" ).get ) 
println( "Wyoming: " + stateCapitals.get( "Wyoming" ).getOrElse( "Oops!" ) ) 
println( "Unknown: " + stateCapitals.get( "Unknown" ).getOrElse( "Oops2!" ) ) 


Get the capitals wrapped in Options:
Alabama: Some(Montgomery)
Wyoming: Some(Cheyenne)
Unknown: None

---
Get the capitals themselves out of the Options:
Alabama: Montgomery
Wyoming: Cheyenne
Unknown: Oops2!


stateCapitals: scala.collection.immutable.Map[String,String] = Map(Alabama -> Montgomery, Alaska -> Juneau, Wyoming -> Cheyenne)


In [30]:
val a: Option[String] = Some("1024")
val b: Option[String] = None
a.map(_.toInt)
//res0: Option[Int] = Some(1024)
b.map(_.toInt)
//res1: Option[Int] = None,不會拋出exception

a.getOrElse("2048") //1024
b.getOrElse("2048") //2048


val x: Seq[String] = Seq("1", "2", "3", null, "4")
// x.map(_.toInt) -> 會錯誤, 如果忘了檢查編譯器是看不出來的,只能在跑的時候拋異常
x.filter(_ != null).map(_.toInt)

// option幫助你把錯誤扼殺在編譯階段 而 flatMap則可以在過濾空值的同時將option恢復為原始數據.
val y: Seq[Option[String]] = Seq(Some("1"), Some("2"), Some("3"), None, Some("4"))
y.flatMap(_.map(_.toInt))

a: Option[String] = Some(1024)
b: Option[String] = None
x: Seq[String] = List(1, 2, 3, null, 4)
y: Seq[Option[String]] = List(Some(1), Some(2), Some(3), None, Some(4))
res21: Seq[Int] = List(1, 2, 3, 4)


In [32]:
/*
def get(index:Int):Int = {
	if(x < 0) throw new Exception(...)
	else ....
if語句是表達式，有返回值，必然有返回值類型，如果x < 0，拋出異常，返回值的類型為Nothing，
Nothing也是Int的子類，所以，if表達式的返回類型為Int， get方法的返回值類型也為Int。 

*/

 ### Range 用法
* 支持類型創建範圍：Int，Long，Float，Double，Char，BigInt和BigDecimal。

In [42]:
println(1 to 5 )
println(1 until 5 )
println(1 to 20 by 4 )

val v5 = 1.1f to 10.3f by 0.5f // Float with an interval < 1
val v7 = 'a' to 'g' by 1 // Char 

Range(1, 2, 3, 4, 5)
Range(1, 2, 3, 4)
Range(1, 5, 9, 13, 17)


v5: scala.collection.immutable.NumericRange[Float] = NumericRange(1.1, 1.6, 2.1, 2.6, 3.1, 3.6, 4.1, 4.6, 5.1, 5.6, 6.1, 6.6, 7.1, 7.6, 8.1, 8.6, 9.1, 9.6, 10.1)
v7: scala.collection.immutable.NumericRange[Char] = NumericRange(a, b, c, d, e, f, g)


### Scala中==,eq與equals的區別
* Scala使用==作為邏輯等式，但它調用equals方法。
* 當您想要比較引用，但不測試邏輯時，可以使用新的方法 eq。
* 推薦使用==方法，因為如果比較值為null的情況下，調用equals方法是會報錯的，而==方法事先為我們檢查了是否為null，然後在進行相應比較
* 對於Array或者Map對像不能簡單點使用equals進行值比較，要通過sameElements方法

In [71]:
case class Bread(brand:String, price:Int) 


val b1 = Bread("BreadTalk", 50)
val b2 = Bread("BreadTalk", 80)


println(b1 eq b2)
println(b1.brand equals b2.brand)
println(b1.brand == b2.brand)
println("\n---")

val c1 =  new String("test")
val c2 =  new String("test")

println(c1 eq c2)
println(c1 == c2)
println("\n---")

//array 不能直接 == 比較
val a1 = Array("x", "y")
val a2 = Array("x", "y")

println(a1 eq a2)
println(a1 == a2)
println(a1.deep == a2.deep) //深度比較
println(a1 sameElements a2)

println("\n---")

//Map 可以直接 == 比較
val m1 = Map(1->"x", 2->"y")
val m2 = Map(1->"x", 2->"y")
println(m1 eq m2)
println(m1 == m2)
println(m1 sameElements m2)

false
true
true

---
false
true

---
false
false
true
true

---
false
true
true


defined class Bread
b1: Bread = Bread(BreadTalk,50)
b2: Bread = Bread(BreadTalk,80)
c1: String = test
c2: String = test
a1: Array[String] = Array(x, y)
a2: Array[String] = Array(x, y)
m1: scala.collection.immutable.Map[Int,String] = Map(1 -> x, 2 -> y)
m2: scala.collection.immutable.Map[Int,String] = Map(1 -> x, 2 -> y)


---
## try catch 

* try/finally結構在Scala和Java中也是一樣的  


try {  

    throw newException("some exception...")   
    
} finally{  

    println("This will always be printed")   
    
}

---

* 因為Scala中的try/catch是一個表達式，所以可以在try / catch中包裝調用，並在調用失敗時分配默認值。  

try {   

    file.write(stuff) 
} catch{   

    case e:java.io.IOException => // handle IO Exception 
    case n:NullPointerException => // handle null pointer 
}

---
### Scala 函數  


* Scala 函數def格式如下：

    def functionName ([参数列表]) : [return type]  
    

* Scala 函數定義格式如下：  
    方法定義由一個def 關鍵字開始，緊接著是可選的參數列表，一個冒號"：" 和方法的返回類型，  
    一個等於號"=" (相當於分隔符)，最後是方法的主體。
    
    def functionName ([参数列表]) : [return type] = {
   function body
   return [expr]
    }  
    
    
* 如果函數沒有返回值，可以返回為Unit，這個類似於Java的void

In [5]:
//注意
//我們還可以包括可選的返回類型，如下所示。
def hello():String = {"Hello World!"} 

//我們可以從方法體中完全刪除括號。
def hello2() = "Hello World!" 

//我們也可以從方法簽名中刪除括號。
def hello3 = "Hello World!" 

hello: ()String
hello2: ()String
hello3: String


### Scala函數編程

* Scala允許我們將函數表達為文字。

    函數文字允許我們有一個函數類型的表達式，我們可以寫一個短格式，而不聲明它的名稱。  


* 函數類型可以是以下之一：

    1. 可以為其分配函數的變量或參數的類型
    2. 採用函數參數的高階函數的參數
    3. 返回函數的高階函數的結果類型
    4. 函數常量以括號括起來的逗號分隔的參數列表開頭，後跟箭頭和函數體。
       函數常量也稱為匿名函數。
         
         
* 函數常量被實例化為稱為函數值的對象。  
    函數對象擴展FunctionN traits中的一個，例如Function0，Function1等等直到Function22。  
    對於具有兩個參數的函數，編譯器選擇Function2作為底層類型。對於具有3個參數的函數，編譯器選擇Function3，對於具有4個參數的函數，Function4等。

In [6]:
//考慮一個add函數:
val add = (x: Int, y: Int) => x + y

//使用函數常量，您可以定義添加函數，如下所示：
(x: Int, y: Int) => x + y

add: (Int, Int) => Int = <function2>
res2: (Int, Int) => Int = <function2>


In [7]:
// 因為函數值是一個對象，它可以存儲在一個變量中，它可以使用括號函數調用來調用，如下所示：
val add = (a: Int, b: Int) => a + b
   println(add(1, 2));

3


add: (Int, Int) => Int = <function2>


### Scala函數是對象。

Scala函數是對象。可以  
1. 分配給變量，  
2. 作為參數傳遞給另一個函數  
3. 作為其他函數的值返回。  
4. 將函數作為參數或返回函數的函數 --> 高階函數(Higher-Order Function)。  

In [9]:
// 我們現在可以將該函數常量分配給一個變量：
val doubler = (i: Int) => { i * 2 }

//doubler 是使用關鍵字 val 創建並分配給變量的函數。 --> Function1 trait的一個實例
doubler(2)


//高階函數
//函數 f 和 值 v 作為參數，而函數 f 又調用了參數 v

def apply(f: Int => String, v: Int) = f(v)
def layout[A](x: A) = "[" + x.toString() + "]"

println( apply( layout, 10) )

[10]


doubler: Int => Int = <function1>
apply: (f: Int => String, v: Int)String
layout: [A](x: A)String


### 函數作為參數

In [9]:
// function param函數接受兩個Int並返回一個 Int 。
def operation(functionparam:(Int, Int) => Int) {
    println(functionparam(4,4))
}

val add = (x: Int, y:Int) => { x + y }
operation(add)

8


operation: (functionparam: (Int, Int) => Int)Unit
add: (Int, Int) => Int = <function2>


### Scala的解釋器在解析函數參數(function arguments)時有兩種方式：
* 傳值調用（call-by-value）：先計算參數表達式的值，再應用到函數內部；
* 傳名調用（call-by-name）：將未計算的參數表達式直接應用到函數內部  

在進入函數內部前，  
傳值調用 **調用方法就已經將參數表達式的值計算完畢**，    
傳名調用 **是在函數內部進行參數表達式的值計算的**。
  
這就造成了一種現象，每次使用傳名調用時，解釋器都會計算一次表達式的值。

In [6]:
//call-by-value

def time(): Long = {
  println("In time()")
  System.nanoTime
}

def exec(t: Long): Long = {
  println("Entered exec, calling t ...")
  println("t = " + t)
  println("Calling t again ...")
  t
}

//exec 在調用 time 的時候 time 已經先算完 所以 t = constant
println(exec(time()))


In time()
Entered exec, calling t ...
t = 192027613503002
Calling t again ...
192027613503002


time: ()Long
exec: (t: Long)Long


In [7]:
def time() = {
  println("Entered time() ...")
  System.nanoTime
}

// `t` is now defined as a by-name parameter
def exec(t: => Long) = {
  println("Entered exec, calling t ...")
  println("t = " + t)
  println("Calling t again ...")
  t
}

// t 會在函式裡面才調用 所以兩次值會不一樣
println(exec(time()))

Entered exec, calling t ...
Entered time() ...
t = 192079117520623
Calling t again ...
Entered time() ...
192079117582458


time: ()Long
exec: (t: => Long)Long


### Scala 函數- 可變參數
* Scala 允許你指明函數的最後一個參數可以是重複的，即我們不需要指定函數參數的個數，可以向函數傳入可變長度參數列表。  

    Scala 通過在參數的類型之後放一個星號來設置可變參數(可重複的參數)。

In [8]:
def printStrings( args:String* ) = {
  var i : Int = 0;
  for( arg <- args ){
     println("Arg value[" + i + "] = " + arg );
     i = i + 1;
  }
}

printStrings("Runoob", "Scala", "Python");

Arg value[0] = Runoob
Arg value[1] = Scala
Arg value[2] = Python


printStrings: (args: String*)Unit


### scala 偏應用函數

Scala 偏應用函數是一種表達式，你不需要提供函數需要的所有參數，只需要提供部分，或不提供所需參數

In [10]:
def sum(a: Int, b: Int, c: Int) = a + b + c  

//Scala編譯器以偏應用函數表達式，sum _，實例化一個帶三個缺失整數參數的函數值，並把這個新的函數值的索引賦給變量a。
//當你把這個新函數值應用於三個參數之上時，它就轉回頭調用sum，並傳入這三個參數
val a = sum _  

sum: (a: Int, b: Int, c: Int)Int
a: (Int, Int, Int) => Int = <function3>


In [11]:
//a的變量指向一個函數值對象。
//Scala編譯器把表達式a(1,2,3)翻譯成對函數值的apply方法的調用，傳入三個參數 1，2，3。
a( 1 ,  2 ,  3 )  
a.apply( 1 ,  2 ,  3 )  

res8: Int = 6


In [12]:
//提供了第一個和最後一個參數給sum，但中間參數缺失
val b = sum( 1 , _: Int,  3 )  
//這個例子裡，b.apply調用了sum(1,5,3)。
b( 5 )  

b: Int => Int = <function1>
res9: Int = 9


### Scala 函數柯里化(Currying)  
柯里化(Currying)指的是將原來接受兩個參數的函數變成新的接受一個參數的函數的過程。  
新的函數返回一個以原有第二個參數為參數的函數。

In [13]:
def sum(a: Int)(b: Int) = a + b
println(sum(1)(2)) 


val s = sum(5)_
println(s(2))  

3
7


sum: (a: Int)(b: Int)Int
s: Int => Int = <function1>


### Scala 閉包
閉包是一個函數，返回值依賴於聲明在函數外部的一個或多個變量。

In [16]:
var factor = 3  
val multiplier = (i:Int) => i * factor
println( "muliplier(1) value = " +  multiplier(5) ) 

muliplier(1) value = 15


factor: Int = 3
multiplier: Int => Int = <function1>


## scala underscore 用法 (In function)
---
### case 1: using underscore in input argument

* use underscore for the argument of a lambda expression when **the input argument is not going to used in the body of the lambda expression** and thus you use the underscore as a placeholder instead of declaring a input argument for the lambda expression

In [4]:
List(1,2,3,4).foreach(_ => println("*" * 10))
println("-----")
List(1,2,3,4).foreach(x => println("*" * 10))


**********
**********
**********
**********
-----
**********
**********
**********
**********


### case 2: using underscore in body of lambda expression

* when **underscore is used in body of lambda expression** it refers to the input argument. You can use the underscore in this fashion if the input is going to be referred only once.

In [9]:
List(1,2,3,4).foreach(println(_))
println("-----")
//等價於
List(1,2,3,4).foreach(x => println(x))

1
2
3
4
-----
1
2
3
4


In [12]:
List(1,2,3,4).foreach(x => println("*" * x))

//等價於下式 但下式會有type mismatch的問題

//List(1,2,3,4).foreach(println("*" * _)) 

/*
error: type mismatch;
 found   : Unit
 required: Int => ?
       List(1,2,3,4).foreach(println("*" * _))
                                    ^
*/

println("-----")
List(1,2,3,4).map("*"*_).foreach(println _)

*
**
***
****
-----
*
**
***
****


### case 3: refer to unapplied methods

* unapplied function here means getting a reference to a function object which can be executed later on demand 

In [31]:
List(1,2,3,4).foreach(_ => println(("*" * _)))

<console>: 26: error: missing parameter type for expanded function ((x$2) => "*".$times(x$2))

In [32]:
"*" * _

res26: Int => String = <function1>


In [35]:
val x  = List(1,2,3,4).map(_ =>  "*" *_ )
x(0).apply(3)

x: List[Int => String] = List(<function1>, <function1>, <function1>, <function1>)
res28: String = ***


In [38]:
List(1,2,3,4).map(x =>  "*" * x )
List(1,2,3,4).map("*" * _ )

res31: List[String] = List(*, **, ***, ****)


### **you cannot mix case 1 and case 2** 
* you can use the underscore either in input argument **or** in the body of the lambda function but not in both. 
* since you are mixing both the cases you are unexpectedly using case 3 of underscore

In [45]:
List(1,2,3,4).foreach(_ => println(("*" * _)))

<function1>
<function1>
<function1>
<function1>


---
<code>
import scala._    // Wild card -- all of Scala is imported
import scala.{ Predef => _, _ } // Exception, everything except Predef
def f[M[_]]       // Higher kinded type parameter
def f(m: M[_])    // Existential type
_ + _             // Anonymous function placeholder parameter
m _               // Eta expansion of method into method value
m(_)              // Partial function application
_ => 5            // Discarded parameter
case _ =>         // Wild card pattern -- matches anything
val (a, _) = (1, 2) // same thing
for (_ <- 1 to 10)  // same thing
f(xs: _*)         // Sequence xs is passed as multiple parameters to f(ys: T*)
case Seq(xs @ _*) // Identifier xs is bound to the whole matched sequence
var i: Int = _    // Initialization to the default value
def abc_<>!       // An underscore must separate alphanumerics from symbols on identifiers
t._2              // Part of a method name, such as tuple getters
</code>  

---

### Scala  字符串
* String 對像是不可變的，如果你需要創建一個可以修改的字符串，可以使用String Builder 類

In [19]:
val buf = new StringBuilder;
buf += 'a'
buf ++= "bcdef"
buf.append("_cat ")
println( "buf is : " + buf.toString );

buf is : abcdef_cat 


buf: StringBuilder = abcdef_cat


In [20]:
// Use this as a custom format string.
val format = "Cats: @, dogs: %"
println(format)

// Create a new StringBuilder.
val builder = StringBuilder.newBuilder
val cats = 10
val dogs = 20

// Loop over the format string.
// ... Append values in place of format characters.
for (i <- format.indices) {
    if (format(i) == '@') {
        builder.append(cats)
    }
    else if (format(i) == '%') {
        builder.append(dogs)
    }
    else {
        builder.append(format(i))
    }
}

// Print the results.
val result = builder.toString()
println(result)

Cats: @, dogs: %
Cats: 10, dogs: 20


format: String = Cats: @, dogs: %
builder: StringBuilder = Cats: 10, dogs: 20
cats: Int = 10
dogs: Int = 20
result: String = Cats: 10, dogs: 20


In [21]:
//字符串長度 可以使用length() 方法來獲取
var palindrome = "www.runoob.com";
var len = palindrome.length();

palindrome: String = www.runoob.com
len: Int = 14


### 字符串連接  
* String 類中使用concat() 方法來連接兩個字符串：  
string1.concat(string2);  
同樣你也可以使用加號(+)來連接
---

### printf() 
String 類中你可以使用printf() 方法來格式化字符串並輸出，  
String format() 方法可以返回String 對象而不是PrintStream 對象

In [22]:
var floatVar = 12.456
var intVar = 2000
var stringVar = "test"
var fs = printf("floatVar " +"%f,intVar %d, stringVar " +" %s", floatVar, intVar, stringVar)
println(fs)

floatVar 12.456000,intVar 2000, stringVar  test()


floatVar: Double = 12.456
intVar: Int = 2000
stringVar: String = test
fs: Unit = ()


---
### Scala 類  
* 類是對象的抽象，而對像是類的具體實例。類是抽象的，不佔用內存，而對像是具體的，佔用存儲空間。  
  類是用於創建對象的藍圖，它是一個定義包括在特定類型的對像中的方法和變量的軟件模板。  
* 我們可以使用new 來實例化類，並訪問類中的方法和變量 (通常Scala中的類不聲明為public)


####  結構說明 :

<code>

class Name [private] ( **Main Constructor 的參數就直接定義在這** ) {   
從頭開始定義的還是Main Constructor的一部分

**auxiliary constructor** 使用this來創建一個或多個輔助 constructor
但是第一行要調用主constructor或是其他輔助constructor
e.g. def this(name: String){
    this() -> 調用主constructor
    this.name = name
    }
}
    
</code>  

---


### Scala 繼承   
* Scala繼承一個基類跟Java很相似，只多了兩點限制：
    1. 字段和重寫方法需要override關鍵字
    2. 只有主構造函數才可以往基類(父類)的構造函數里寫參數  
* 繼承會繼承父類的所有屬性和方法，Scala 只允許繼承一個父類

In [22]:
class Shape {
   def area:Double = 0.0
}


class Rectangle(val width:Double,val height:Double) extends Shape {
   override def area:Double = width*height
}
class Circle(val radius:Double) extends Shape {
   override def area:Double = math.Pi*radius*radius
}

def draw(s:Shape){println(s.area)}

//現在，考慮對這個函數的兩個調用，像這樣：
val circle = draw(new Circle(3))
val rectangle = draw(new Rectangle(2,3))

28.274333882308138
6.0


defined class Shape
defined class Rectangle
defined class Circle
draw: (s: Shape)Unit
circle: Unit = ()
rectangle: Unit = ()


In [18]:
import java.io._

class Point(xc: Int, yc: Int) {
   var x: Int = xc
   var y: Int = yc

   def move(dx: Int, dy: Int) {
      x = x + dx
      y = y + dy
      println ("x : " + x);
      println ("y : " + y);
   }
}

class Location(val xc: Int, val yc: Int, val zc :Int) extends Point(xc, yc){
    var z: Int = zc
    
    /*move前面不能放 overide 會出現底下錯誤 
    error: method move overrides nothing.
    the super classes of class Location contain the following, non final members named move
    不能重載其父類，是因為參數類型不一致
    */
    def move(dx: Int, dy: Int, dz: Int) {
      x = x + dx
      y = y + dy
      z = z + dz
      println ("x  : " + x);
      println ("y  : " + y);
      println ("z  : " + z);
   }
}

val loc = new Location(10, 20, 15);
loc.move(10, 10, 5);

x  : 20
y  : 30
z  : 20


import java.io._
defined class Point
defined class Location
loc: Location = Location@6dcd73c5


### Scala  case class  

* Scala可以創建具有常見內容填充的類。  
  大多數時候，當我們定義一個類時，我們必須編寫toString，hashCode和equals方法。  
  Scala提供了用於填充這些空白的case類機制，以及支持模式匹配。  
  

* case類提供與普通類相同的功能，但編譯器生成toString，hashCode和equals方法，可以覆蓋。  
  

*  可以在不使用 new 的情況下實例化Case類。  
  

* default序列化的實現。  
  

* 從scala.Product自動繼承的少量功能  
  

* 默認情況下，case類的構造函數中的所有參數都成為case類的屬性。

In [40]:
case class Stuff(name:String, age: Int)

val s = Stuff("David", 45) 

println(s.toString)
println(s.age.toString)

println("---")

println(s == Stuff("David",45))
println(s == Stuff("David",43))

Stuff(David,45)
45
---
true
false


defined class Stuff
s: Stuff = Stuff(David,45)


### case class最重要的功能，就是可以作為match運算式的case比對條件

In [41]:
case class Point(x: Int, y: Int)

def go(p: Point) = p match {
    case Point(0, 0) => "start"
    case Point(10, 10) => "End"
    case _ => "run to " + p
}

println(go(Point(0, 0)))
println(go(Point(0, 1)))    // Point(0,1)
println(go(Point(2, 3)))    // Point(2,3)
println(go(Point(10, 10)))  

start
run to Point(0,1)
run to Point(2,3)
End


defined class Point
go: (p: Point)String


### Scala 對象

* 在Scala 中，是沒有static 這個東西的，但是它也提供了單例模式的實現方法，那就是使用關鍵字object。
* Scala 中使用單例模式時，除了定義的類之外，還要定義一個同名的object對象，它和類的區別是，**object對像不能帶參數**。

---

我們可以將singleton對像用於許多目的，包括收集相關的實用程序方法，或者定義一個到Scala應用程序的入口點。  

有兩種方法為應用程序創建啟動點：使用正確定義的main方法定義對象，或定義對像或擴展App特性。  

對於第二種方法，定義一個擴展App trait的對象，如下所示：  

object Main extends App {
    println( "Hello, world" )
}   
  
  
object Test {
 def main(args: Array[String]) { }
 }



### Scala 伴生對象

* 當單例對象與某個類共享同一個名稱時，你必須在同一個源文件裡定義類和它的伴生對象：companion object 和 companion class。
* 類和它的伴生對象**可以互相訪問其私有成員**  
- 在java中被宣告為stastic 的方法和成員可以直接把class作為命名空間 --> classname.成員或是函數 來呼叫
- 而在scala 他沒有static 所以 利用伴生對象的話
    1. class  的成員 - 實例成員 - 屬於對象的, 每個實例都會有自己的副本
    2. object 的成員 - static成員 - 屬於類,會被所有實例共享

---

### Scala Traits

* Traits就像Java中的接口(Interface)，與接口不同的是，它還可以定義屬性和方法的實現。  
* 一般情況下Scala的類只能夠繼承單一父類，但是如果是Trait(特徵) 的話就可以繼承多個，從結果來看就是實現了多重繼承。
* 在Scala中，當一個類從trait繼承時，它實現trait的接口，並繼承trait中包含的所有代碼。
    
       
       
  
* 在Scala中，traits可以繼承類。當一個類繼承一個trait作為其父類時，也使用關鍵字extends。
* 即使當類使用with關鍵字在其他traits中混合時，也使用關鍵字extends。
    1. 如果是待聲明的類沒有擴展其他類只是或入了一些trait那麼必須用extends關鍵字，並把它用在第一個trait之前而其它的trait之前用with關鍵字
    2. 如果在實例化某一個類型的時候想要混入某一個trait，這個時候要用with關鍵字
* 此外，當一個trait是另一個trait或類的子對象時使用extends。  

In [6]:
class foo {
    private val secret = 2
}

object foo {
    
    def get_private(f:foo) = {println(s"find the secret number : ${f.secret}" )}
}

//---

class boo {
    
    def printObj { println(s"I can see ${boo.obj}") }
}


object boo {
    private val obj = "boo's object"
}
//---

println("---object access the class---")
val f = new foo
foo.get_private(f)

println("---class access the object---")
val b = new boo
b.printObj



---object access the class---
find the secret number : 2
---class access the object---
I can see boo's object


defined class foo
defined object foo
defined class boo
defined object boo
f: foo = foo@3741d4ce
b: boo = boo@1ec04af9


In [4]:
// 私有構造方法
class Marker private(val color:String) {
    
    ////在scala中，this是一個關鍵字，用於引用當前對象。可以使用this關鍵字調用實例變量，方法，構造函數。
    println("create " + this) 
    override def toString(): String = "color mark ："+ color

}

// 伴生對象，與類共享名字，可以訪問類的私有屬性和方法
object Marker{

    private val markers: Map[String, Marker] = Map(
      "red" -> new Marker("red"),
      "blue" -> new Marker("blue"),
      "green" -> new Marker("green")
    )

    def apply(color:String) = {
      if(markers.contains(color)) markers(color) else null
    }

    def getMarker(color:String) = { 
      if(markers.contains(color)) markers(color) else null
    }
    def main(args: Array[String]) { 
        println(Marker("red"))  
        // 單例函數調用，省略了.(點)符號
        println(Marker getMarker "blue")  
    }
}

println("****")
println(Marker("red"))
println(Marker getMarker "blue")

/*
create color mark ：red
create color mark ：blue
create color mark ：green
color mark ：red
color mark ：blue

*/

****

defined class Marker
defined object Marker


In [8]:
// extends & with
trait flying {
    def fly() = println("flying")
}

trait gliding {
    def gliding() = println("gliding")
}


class Vehicle (speed : Int){
    val mph :Int = speed
    def race() = println("Racing")
}
class Car (speed : Int) extends Vehicle(speed) {
    override val mph: Int= speed
    override def race() = println("Racing Car")

}
class Bike(speed : Int) extends Vehicle(speed) {
    override val mph: Int = speed
    override def race() = println("Racing Bike")
}

class AirCraft(speed : Int) extends Vehicle(speed) with flying with gliding{
    override val mph: Int = speed
    override def race() = println("Racing")
    override def fly() = println("Flying")
    override def gliding() = println("Gliding")
}

defined trait flying
defined trait gliding
defined class Vehicle
defined class Car
defined class Bike
defined class AirCraft


In [4]:
trait Shape {
    def area :Double
}
object Shape {
    private  class Circle(radius: Double) extends Shape{
        override val area = 3.14*radius*radius
    }
    
    private  class Rectangle (height: Double, length: Double) extends Shape{
        override val area = height * length
    }
    
    def apply(height :Double , length :Double ) : Shape = new Rectangle(height,length)
    def apply(radius :Double) : Shape = new Circle(radius)

}

object Main extends App {
    val circle_ins = Shape(2)
    println(circle_ins.area)
    val rectangle_ins = Shape(2,3)
    println(rectangle_ins.area)
}

defined trait Shape
defined object Shape
defined object Main


### scala package object
* 您希望在包級別上使Scala函數，字段和其他代碼可用，而不需要類或對象。
* By convention, put your code in a file named package.scala in the directory where you want your code to be available.   


1. 想要作用在 package alvinalexander.myapp.model 都可以用到package object  

example :  

        +-- alvinalexander  
            +-- myapp   
                +-- model  
                    +-- package.scala 
                    (package.scala裡面用到的宣告是
                    package alvinalexander.myapp
                    package object model {--}
                    則在alvinalexander.myapp.model這層內裡面都吃得到package object的函數，字段和其他代碼)  
2. 假設是這樣

        +-- alvinalexander  
            +-- myapp   
                +-- model  
                    +-- dir(裡面有其他files)  
                        +-- test
                    +-- package.scala                  
                        (則在此範例中 由於scala 2.8的chained package的scope改變,在test之中
                        package alvinalexander.myapp.model.dir(x) --> 不會被視為同一個nested package底下
                        正確會是底下 :
                        package alvinalexander.myapp.model
                        package dir )

                    

### Scala 資料結構  

* 所有的集合類都可以在包 scala.collection中找到   
    (scala.collection.mutable，scala.collection.immutable，scala.collection.generic)   

### 序列的操作有以下幾種，如下表所示：
    1. 序列的length方法是collection的size方法的別名。lengthCompare方法可以比較兩個序列的長度，即便其中一個序列長度無限也可以處理。
    2. 索引檢索操作（indexOf、lastIndexOf、indexofSlice、lastIndexOfSlice...）用於返回等於給定值或滿足某個謂詞的元素的索引。
    3. 加法運算（+:，:+，padTo）用於在序列的前面或者後面添加一個元素並作為新序列返回。
    4. 更新操作（updated，patch）用於替換原序列的某些元素並作為一個新序列返回。
    5. 排序操作（sorted, sortWith, sortBy）根據不同的條件對序列元素進行排序。
    6. 反轉操作（reverse, reverseIterator, reverseMap）用於將序列中的元素以相反的順序排列。
    7. 比較（startsWith, endsWith, contains, containsSlice, corresponds）用於對兩個序列進行比較，或者在序列中查找某個元素。
    8. 多集操作（intersect, diff, union, distinct）用於對兩個序列中的元素進行類似集合的操作，或者刪除重複元素。  
       
       
* 常用線性序列有scala.collection.immutable.List和scala.collection.immutable.Stream。
* 常用索引序列有scala.Array scala.collection.mutable.ArrayBuffer     


### 緩衝器的操作
* Buffers是可變序列一個重要的種類。它們不僅允許更新現有的元素，而且允許元素的插入、移除和在buffer尾部高效地添加新元素。
* buffer支持的主要新方法有：
    1. 用於在尾部添加元素的+=和++=；用於在前方添加元素的+=: 和 ++=:；
    2. 用於插入元素的insert和insertAll；以及用於刪除元素的 remove和-=。  
       
       
* ListBuffer和ArrayBuffer是常用的buffer實現。顧名思義，ListBuffer依賴列表（List），支持高效地將它的元素轉換成列表。而ArrayBuffer依賴數組（Array），能快速地轉換成數組。

---
### Scala array 
    - 由相同類型的元素的集合組成的數據結構。
    - 元素與索引相關聯，索引通常為整數，用於訪問或替換特定元素。
    - 有兩種方法來定義數組：指定元素的總數，然後將值分配給元素，或者我們可以一次指定所有值。

In [35]:
var books = new Array[String](3) 
var books2 = new Array[String](4) 

books: Array[String] = Array(null, null, null)
books2: Array[String] = Array(null, null, null, null)


In [36]:
books = Array("Scala", "Java", "Groovy") 
books2 = Array("C", "C++", "R", "python") 
books(1) = "C#" 

books: Array[String] = [Ljava.lang.String;@27cc8543
books2: Array[String] = [Ljava.lang.String;@6579ae9d


In [37]:
books +: books2

res22: Array[java.io.Serializable] = Array(Array(Scala, C#, Groovy), C, C++, R, python)


In [38]:
books ++ books2

res23: Array[String] = Array(Scala, C#, Groovy, C, C++, R, python)


In [39]:
// 2維array
val matrix = Array.ofDim[Int](2,2)
matrix(0)(0) = 1234

matrix: Array[Array[Int]] = Array(Array(1234, 0), Array(0, 0))


In [48]:
//補充: take 操作
val names = Array("spark", "hadoop","kafka","hive", "mesos", "zero", "xyz","marathon")

//takeWhile, 從第一個元素開始判斷，滿足條件，就留下，直到遇到第一個不滿足的條件的元素，就結束循環
//可見，takeWhile 有可能並不是對所有的元素進行操作的
names.takeWhile(_.length > 4 ).foreach{x => print(x + " ")}

println("\n------------------")
//從左邊開始獲取2個元素，
names.take(2).foreach{x => print(x + " ")}

println("\n------------------")

//從右邊開始獲取4個元素，
names.takeRight(4).foreach{x => print(x + " ")}

println("\n------------------")
//filter， 同樣，滿足條件，就留下。是對所有的元素，進行操作的
names.filter(_.length > 4).foreach{x => print(x + " ")}//將"xyz"元素，過濾掉了

spark hadoop kafka 
------------------
spark hadoop 
------------------
mesos zero xyz marathon 
------------------
spark hadoop kafka mesos marathon 

names: Array[String] = Array(spark, hadoop, kafka, hive, mesos, zero, xyz, marathon)


### Scala列表
* 在Scala列表中，所有元素都具有類似數組的類型，但與數組不同，列表的元素不能通過賦值進行更改。
* 具有類型T的元素的列表被寫為List [T]。
* 列表可以使用tail Nil和::定義。
  1. Nil也表示空列表。
  2. 列表上的操作可以用head和tail方法表示，其中head返回列表的第一個元素，tail返回一個由除第一個元素之外的所有元素組成的列表
  3. 可以使用:::運算符或List.:::()方法或List.concat()方法來連接兩個或多個列表。
  4. List.tabulate() 方法是通過給定的函數來創建列表。

In [23]:
val books: List[String] = List("Scala", "Groovy", "Java") 
books(1) = "R"

<console>: 29: error: value update is not a member of List[String]

In [28]:
// 1 :: 2 :: Nil  --> a list
// list1 ::: list2  --> concatenation of two lists

val books = "Scala" :: ("Groovy" :: ("Java" :: Nil)) 

books: List[String] = List(Scala, Groovy, Java)


In [40]:
 val mul = List.tabulate( 4,5 )( _ * _ )      

mul: List[List[Int]] = List(List(0, 0, 0, 0, 0), List(0, 1, 2, 3, 4), List(0, 2, 4, 6, 8), List(0, 3, 6, 9, 12))


### Scala Set(集合)是沒有重複的對象集合，所有的元素都是唯一的。
* Scala 集合分為可變的和不可變的集合。
* 默認情況下，Scala使用的是不可變集合，如果你想使用可變集合，需要引用scala.collection.mutable.Set包。  
  
  
1. 你可以使用++運算符或Set.++()方法來連接兩個集合。如果元素有重複的就會移除重複的元素
2. 你可以使用Set.min方法來查找集合中的最小元素，使用Set.max方法查找集合中的最大元素
3. 你可以使用Set.&方法或Set.intersect方法來查看兩個集合的交集元素

In [41]:
val num1 = Set(5,6,9,20,30,45)
val num2 = Set(50,60,9,20,35,55)
println( "num1.&(num2) : " + num1.&(num2) )

num1.&(num2) : Set(20, 9)


num1: scala.collection.immutable.Set[Int] = Set(5, 20, 6, 9, 45, 30)
num2: scala.collection.immutable.Set[Int] = Set(20, 60, 9, 35, 50, 55)


### Buffer 類操作
* Buffers是可變序列一個重要的種類。它們不僅允許更新現有的元素，而且允許元素的插入、移除和在buffer尾部高效地添加新元素。
* buffer支持的主要新方法有：用於在尾部添加元素的+=和++=；用於在前方添加元素的+=: 和 ++=:；
* 用於插入元素的insert和insertAll；以及用於刪除元素的 remove和-=。

In [47]:
import scala.collection.mutable.ArrayBuffer

var buf = ArrayBuffer.empty[Int]

buf += 1
buf += 10
buf.toArray

import scala.collection.mutable.ArrayBuffer
buf: scala.collection.mutable.ArrayBuffer[Int] = ArrayBuffer(1, 10)
res29: Array[Int] = Array(1, 10)


### scala Stream（流） --> immutable

* 流Stream與List很相似，只不過其中的每一個元素都經過了一些簡單的計算處理。  
    也正是因為如此，stream結構可以無限長。只有那些被要求的元素才會經過計算處理，除此以外stream結構的性能特性與List基本相同。

* 鑑於List通常使用:: 運算符來進行構造，stream使用外觀上很相像的#::。
* stream被特別定義為懶惰計算

In [56]:
//if 需要在50個隨機數中找到前兩個可以被3整除的數字。

import util.Random
def randomList = (1 to 50).map(_ => Random.nextInt(100)).toList

def isDivisibleBy3(n: Int) = {
  val isDivisible = n % 3 == 0
  println(f"$n $isDivisible")
  isDivisible
}

//isDivisibleBy3被調用了50次，找到了遠多於兩個的能被3整除的數字，但是最後我們只關心其中前兩個結果。
randomList.filter(isDivisibleBy3).take(2)

70 false
59 false
96 true
91 false
43 false
82 false
86 false
14 false
23 false
9 true
46 false
75 true
13 false
50 false
61 false
90 true
96 true
97 false
51 true
51 true
13 false
93 true
77 false
68 false
17 false
88 false
22 false
40 false
34 false
26 false
97 false
43 false
65 false
63 true
44 false
54 true
70 false
80 false
78 true
42 true
55 false
47 false
34 false
28 false
74 false
64 false
93 true
67 false
75 true
40 false


import util.Random
randomList: List[Int]
isDivisibleBy3: (n: Int)Boolean
res35: List[Int] = List(96, 9)


In [57]:
//Stream利用了惰性求值（lazy evaluation），或者也可以稱之為延遲執行
randomList.toStream.filter(isDivisibleBy3).take(2).toList

4 false
56 false
7 false
34 false
8 false
84 true
84 true


res36: List[Int] = List(84, 84)


### 說明
* List(1,2,3)會構造一個容器，容器中包含數據
* List(1,2,3).filter(n=>n>1)會構造出一個新的容器，其中包含2和3，這兩塊具體的數據
* List(1,2,3).filter(n=>n>1).take(1)會把上一步中構造成的容器中的第一塊數據取出，放入一個新容器
---
* Stream(1,2,3)也會構造一個容器，但是這個容器中不包含數據，它包含能夠生產數據的算法
* Stream(1,2,3).filter(n=>n>1)也會構造出一個新的容器，這個容器中所包含的仍然是算法
* Stream(1,2,3).filter(n=>n>1).take(1)會把上一步中構造成的算法容器中的第一個算法取出，放入一個新容器
* Stream(1,2,3).filter(n=>n>1).take(1).toList終於把上面所有步驟構造出的算法執行了，從而得到了最終想要的結果

In [58]:
def fibFrom(a: Int, b: Int): Stream[Int] = a #:: fibFrom(b, a + b)
val fibs = fibFrom(1, 1).take(7)

fibFrom: (a: Int, b: Int)Stream[Int]
fibs: scala.collection.immutable.Stream[Int] = Stream(1, ?)


In [59]:
fibs.toList

res37: List[Int] = List(1, 1, 2, 3, 5, 8, 13)


In [5]:
//if 需要在50個隨機數中找到前兩個可以被3整除的數字。

import util.Random
def randomList = (1 to 50).map(_ => Random.nextInt(100)).toList

def isDivisibleBy3(n: Int) = {
  val isDivisible = n % 3 == 0
  println(f"$n $isDivisible")
  isDivisible
}


def mapBy3(n: Int) = {
  val isDivisible = n * 3
  println(f"$n $isDivisible")
  isDivisible
}

//isDivisibleBy3被調用了50次，找到了遠多於兩個的能被3整除的數字，但是最後我們只關心其中前兩個結果。
randomList.map(mapBy3).take(2)

19 57
85 255
76 228
84 252
70 210
35 105
31 93
18 54
22 66
20 60
45 135
64 192
60 180
55 165
52 156
71 213
6 18
14 42
64 192
25 75
8 24
9 27
19 57
16 48
27 81
3 9
82 246
44 132
30 90
3 9
23 69
66 198
79 237
85 255
90 270
88 264
17 51
91 273
19 57
33 99
3 9
65 195
14 42
2 6
18 54
13 39
98 294
58 174
25 75
88 264


import util.Random
randomList: List[Int]
isDivisibleBy3: (n: Int)Boolean
mapBy3: (n: Int)Int
res3: List[Int] = List(57, 255)


In [6]:
val distData = sc.parallelize(randomList)

distData: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[0] at parallelize at <console>:28


In [8]:
distData.map(mapBy3).take(2)

66 198
36 108


res5: Array[Int] = Array(198, 108)


### Scala 中 implicit 的用法 !  
* **Scala在面對編譯出現類型錯誤時，提供了一個由編譯器自我修復的機制**，編譯器試圖去尋找一個隱式implicit的轉換方法，轉換出正確的類型，完成編譯。(在Scala 中的implicit 定義指編譯器在需要修復類型匹配時可以用來自動插入的定義。)    

    e.g. 如果x+y 類型不匹配，那麼編譯器可能試著使用convert(x) + y， 其中convert 由某個implicit 定義的  
    p.s. 這有點類似一個整數和一個浮點數相加，編譯器可以自動把整數轉換為浮點數。  
   
   
1. **標記規則** : 只有哪些使用implicit 關鍵字的定義, 編譯器才可以選擇它作為隱式變化的候選項
2. **範圍規則** : 編譯器在選擇備選implicit 定義時，只會選取當前作用域的定義, 如果需要使用 -> 引入到當前作用域
3. **一次規則** : 編譯器在需要使用implicit 定義時，只會試圖轉換一次，也就是編譯器永遠不會把x + y 改寫成convert1(convert2(x)) + y。
4. **優先規則** : 編譯器不會在x+y 已經是合法的情況下去調用implicit 規則。
5. **命名規則** : implicit 的名稱只在兩種情況下有用：一是你想在一個方法中明確指明，另外一個是想用名字import到當前作用域  

  
* 編譯器使用implicit 的幾種情況  
    1. **轉換成預期的數據類型** : 比如你有一個方法參數類型是IndexedSeq[Char]，在你傳入String 時，編譯器發現類型不匹配，就檢查當前作用域是否有從String 到IndexedSeq 隱式轉換。
    2. **轉換被方法調用的對象** : 比如“abc”.exist ，”abc”類型為String，本身沒有定義exist 方法，這時編輯器就檢查當前作用域內String 的隱式轉換後的類型是否有exist 方法
    3. **隱含參數** : 隱含參數有點類似是缺省參數，如果在調用方法時沒有提供某個參數，編譯器會查找當前作用域是否有符合條件的implicit 對像作為參數傳入


In [9]:
//轉換成預期的數據類型
val i:Int = 3.5
// 其實可以調用 3.5.toInt

<console>: 25: error: type mismatch;

In [10]:
implicit def doubleToInt(x:Double) = x toInt
val i:Int = 3.5 //val i:Int = doubleToInt(3.5)

doubleToInt: (x: Double)Int
i: Int = 3


In [12]:
//轉換被方法調用的對象

//類 Rational 重載了兩個 +運算，參數類型分別為 Rational 和 Int。
//因此你可以把 Rational 和 Rational 相加，也可以把 Rational 和整數相加。
class Rational (n:Int, d:Int) {
    require(d!=0) //運行時執行某些檢查來驗證某些條件
    private val g =gcd(n.abs,d.abs) //最大公因數
    val numer =n/g //1
    val denom =d/g //2
    override def toString = numer + "/" +denom
    def +(that:Rational)  = // 1/2 + 1/2
      new Rational(
        numer * that.denom + that.numer* denom, // 1*2 + 1*2
        denom * that.denom //2*2 
      )
    def +(i:Int) :Rational = //1/2 + 1
        new Rational(numer +1*denom,denom) // 1 + 1*2, 2
    def * (that:Rational) =
      new Rational( numer * that.numer, denom * that.denom)
    def this(n:Int) = this(n,1)
    private def gcd(a:Int,b:Int):Int =
      if(b==0) a else gcd(b, a % b)
}

defined class Rational


In [15]:
val oneHalf = new Rational(1,2)
oneHalf + oneHalf

oneHalf: Rational = 1/2
res8: Rational = 1/1


In [17]:
oneHalf + 1 // ok!

//但是我們如果使用 1 + oneHalf 會出現什麼問題呢
//此時編譯器在1能夠轉換成Rational 類型才可以編譯過
1 + oneHalf

<console>: 33: error: overloaded method value + with alternatives:

In [18]:
implicit def int2Rational(x:Int) = new Rational(x)
1 + oneHalf

int2Rational: (x: Int)Rational
res11: Rational = 3/2


In [34]:
//隱含參數
class PreferredPrompt(val preference:String)


object Greeter{
    //第二個參數標記為 implicit，表明允許編譯器根據需要自動添加
    def greet(name:String)(implicit prompt: PreferredPrompt) {
        println("Welcome, " + name + ". The System is ready.")
        println(prompt.preference)
    }
}

// 先直接都給(第二個參數不給會出錯 -> implicit 須加在 val bobsPrompt 前面)
val bobsPrompt =new PreferredPrompt("relax> ")
Greeter.greet("Bob")(bobsPrompt)

Welcome, Bob. The System is ready.
relax> 


defined class PreferredPrompt
defined object Greeter
bobsPrompt: PreferredPrompt = PreferredPrompt@58b2b039


In [24]:
object JamesPrefs{
    implicit val prompt=new PreferredPrompt("Yes, master> ")
}
// 一般情況是用 import JamesPrefs._ 即可搞定
Greeter.greet("James")


<console>: 37: error: could not find implicit value for parameter prompt: PreferredPrompt

In [66]:
//隱含參數 2
// 這個函數是求取一個順序列表的最大值。但這個函數有個局限，它要求類型 T 是Ordered[T]的一個子類，
// 因此這個函數無法求一個整數列表的最大值。
def maxListUpBound[T <:Ordered[T]](element:List[T]):T =
    element match {
      case List() =>
        throw new IllegalArgumentException("empty list!")
      case List(x) => x
      case x::rest =>
        val maxRest=maxListUpBound(rest)
        if(x > maxRest) x
        else maxRest
    }

maxListUpBound: [T <: Ordered[T]](element: List[T])T


In [67]:
// gg
maxListUpBound(List(1,5,10,34,23))

<console>: 39: error: inferred type arguments [Int] do not conform to method maxListUpBound's type parameter bounds [T <: Ordered[T]]

In [69]:
//定義一個隱含參數，其類型為一函數類型，可以把一個類型 T轉換成Ordered[T]
def maxListImpParam[T](element:List[T])
                    (implicit orderer:T => Ordered[T]):T =
    element match {
      case List() =>
        throw new IllegalArgumentException("empty list!")
      case List(x) => x //裡面只有一個的話 x = max
      case x::rest => // 拿出第一個元素 + 少了第一個元素的list
        val maxRest=maxListImpParam(rest)
        if(x > maxRest) x
        else maxRest
    }
//編譯在看到 x > maxRest 發現類型不匹配，編譯器不會馬上停止編譯，相反，它會檢查是否有合適的隱含轉換來修補代碼，
//在本例中，它發現orderer 可用。因此編譯器自動改寫為orderer(x)> maxRest。

maxListImpParam: [T](element: List[T])(implicit orderer: T => Ordered[T])T


In [70]:
maxListImpParam(List(1,5,10,34,23))

res41: Int = 34


In [71]:
//其中 <% 為View 限定，也就是說，我可以使用任意類型的 T，只要它可以看成類型Ordered[T]。
def maxList[T <% Ordered[T]](element:List[T]) :T =
    element match {
      case List() =>
        throw new IllegalArgumentException("empty list!")
      case List(x) => x
      case x::rest =>
        val maxRest=maxList(rest)
        if(x > maxRest) x
        else maxRest
    }

maxList: [T](element: List[T])(implicit evidence$1: T => Ordered[T])T


In [72]:
maxList(List(1,5,10,34,23))

res42: Int = 34


In [100]:
Map(1 -> "one", 2 -> "two") map Function.tupled(_ -> _.length)

res61: scala.collection.immutable.Map[Int,Int] = Map(1 -> 3, 2 -> 3)


### Scala 2.8 breakOut
* **the definition of map**: 
<code>
def map[ B, that ] (f : (A) => B)(implicit bf : CanBuildFrom[Repr, B, That]) : That
</code>  
  
  
* **the definition of breakOut** : 
<code>
def breakOut[ From, T, To ] (implicit b : CanBuildFrom[Nothing, T, To]) =
  new CanBuildFrom[From, T, To] {
    def apply(from: From) = b.apply() ; def apply() = b.apply()
  }
</code>  

  
 * 考慮一個例子，  
    * Q : You take a list of strings, transform each string into a tuple (Int, String), and then produce a Map out of it.   
      A : The most obvious way to do that would produce an intermediary List[(Int, String)] collection, and then convert it.
 
    * But : Given that map uses a Builder to produce the resulting collection,       
     **wouldn’t it be possible to skip the intermediary List** and **collect the results directly into a Map**? 
----

**Map[Int,String] = List("London", "Paris").map(x => (x.length, x))(breakOut)**  

----

The type of List("London", "Paris") is List[String], so the types A and Repr defined on TraversableLike are:  
<code>
A = String
Repr = List[String]
</code>   

***
The type for (x => (x.length, x)) is (String) => (Int, String), so the type of B is:  
<code>
B = (Int, String)
</code>   
***
The last unknown type, That is the type of the result of map, and we already have that as well:   

<code>
val map : Map[Int,String] =
So,
That = Map[Int, String]
</code>

---
That means breakOut must, necessarily, return a type or subtype of CanBuildFrom[List[String], 
                                                                                (Int, String), 
                                                                                Map[Int, String]].
